In [2]:
import numpy as np

In [3]:
import matlab
import matlab.engine
eng = matlab.engine.start_matlab()
eng.addpath(r'C:/Users/igorp/OneDrive/Documents/GitHub/Project-Heart/matlab',nargout=0)

In [4]:
import os
from pathlib import Path
import numpy as np
import pyvista as pv
# import project_heart as ph
from project_heart.modules.geometry import Geometry
pv.set_jupyter_backend("pythreejs")
from project_heart.lv import LV
from project_heart.enums import *

In [5]:
# dir = Path("C:/Users/igorp/University of South Florida/Wenbin Mao - Igor/LV_Meshes/Heart_models")
# # dir = Path("C:/Users/igornobrega/USF_Research/University of South Florida/Wenbin Mao - Igor/LV_Meshes/Heart_models")
# files = os.listdir(dir)

# file_id = 3 # 5 has a problem

print("File:", "C:/Users/igorp/OneDrive/Desktop/sample_lv_1.xplt")

lv = LV()
lv.from_xplt("C:/Users/igorp/OneDrive/Desktop/sample_lv_1.xplt", identifier="elemTag", threshold=[0, 1])

plotter = pv.Plotter(lighting='three lights')
plotter.background_color = 'w'
plotter.enable_anti_aliasing()
plotter.add_mesh(lv.mesh, 
                  cmap="Set2", 
                  opacity=1.0, 
                  show_edges=True,
                  ambient=0.2, 
                  diffuse=0.5, 
                  specular=0.5, 
                  specular_power=90,
                  #  smooth_shading=True,
                  )
plotter.enable_shadows()
plotter.show()

File: C:/Users/igorp/OneDrive/Desktop/sample_lv_1.xplt


Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [6]:
lv.identify_base_and_apex_regions()

(array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]))

In [8]:
import tetgen
tet = tetgen.TetGen(lv.get_surface_mesh().triangulate())
tet.tetrahedralize(order=1, 
                   nobisect=True,
                  #  quality=False,
                   mindihedral=100, 
                   minratio=1.1,
                  #  steinerleft=-1
                   )
lv_tet = LV()
lv_tet.from_pyvista_dataset(tet.grid)
lv_tet

In [9]:
lv_tet.get_surface_mesh()

PolyData (0x16e1a9c0100)
  N Cells:	43808
  N Points:	21906
  X Bounds:	-3.948e+01, 3.455e+01
  Y Bounds:	-3.831e+01, 3.370e+01
  Z Bounds:	5.064e-02, 9.641e+01
  N Arrays:	2

In [10]:
lv_tet.mesh

UnstructuredGrid,Information
N Cells,663889
N Points,121906
X Bounds,"-3.948e+01, 3.455e+01"
Y Bounds,"-3.831e+01, 3.370e+01"
Z Bounds,"5.064e-02, 9.641e+01"
N Arrays,0


In [11]:
plotter = pv.Plotter(lighting='three lights')
plotter.background_color = 'w'
plotter.enable_anti_aliasing()
plotter.add_mesh(lv_tet.mesh, 
                  cmap="Set2", 
                  opacity=1.0, 
                  show_edges=True,
                  ambient=0.2, 
                  diffuse=0.5, 
                  specular=0.5, 
                  specular_power=90,
                  #  smooth_shading=True,
                  )
plotter.enable_shadows()
plotter.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [22]:
edges = lv_tet.get_surface_mesh().extract_feature_edges().extract_largest()

In [23]:
edges.plot()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', inten…

In [101]:
eoi = edges.extract_largest().point_data["vtkOriginalPointIds"]
eoi = set(eoi)

In [102]:
cells = lv_tet.cells()[10]
cells

array([[116754, 112350, 121078,  91199],
       [ 82763,  46998,  81104, 108277],
       [ 32086,  39890,  54455,  24201],
       ...,
       [108282,  98487, 116999, 121904],
       [ 16405,  47552,  36799,  16402],
       [121662,  57095,  57597,  48806]], dtype=int64)

In [103]:
cell_map = np.ones(len(cells), dtype=np.int64)
cells_flat = cells.flatten()
for idx, cell in enumerate(cells_flat):
  if cell in eoi:
    cell_map[int(np.floor(idx/4))] = 2

In [93]:
cells_flat

array([116754, 112350, 121078, ...,  57095,  57597,  48806], dtype=int64)

In [104]:
lv_tet.mesh.cell_data["rids"] = cell_map
lv_tet.mesh.set_active_scalars("rids")
lv_tet.mesh.plot()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', inten…

In [13]:
lv_tet.to_vtk("tet.vtk")

In [12]:
tet_nodes = lv_tet.points()
tet_cells = lv_tet.cells(as_json_ready=True)["TETRA"]+1
# tet_doms = np.ones(len(tet_cells), dtype=np.int64)

In [13]:
np.savetxt("nodes.csv", tet_nodes, delimiter=",")
np.savetxt("elems.csv", tet_cells.astype(np.int32), fmt='%i', delimiter=",")

In [105]:
np.savetxt("regionIDs.csv", cell_map, fmt='%i', delimiter=",")

In [ ]:
tet_nodes = lv_tet.points()
tet_cells = lv_tet.cells(as_json_ready=True)["TETRA"]+1
# tet_doms = np.ones(len(tet_cells), dtype=np.int64)

# tet_doms[:2] = 2

tet_mesh = {
  "NODES": matlab.double(tet_nodes), 
  "ELEMENTS": matlab.int64(tet_cells),
  # "DOMAINS": matlab.int64(tet_doms)
  }

In [ ]:
eng.calc_fiber_orientation(
  tet_mesh,
  tet_mesh,
  matlab.double(lv.get_virtual_node(LV_VIRTUAL_NODES.APEX.value)),
  matlab.double(lv.get_virtual_node(LV_VIRTUAL_NODES.BASE.value)),
  -60.0,
  60.0
)

MatlabExecutionError: Output argument "f0" (and possibly others) not assigned a value in the execution with "calc_fiber_orientation" function.


In [ ]:
cells = lv.cells(as_json_ready=True)["TETRA"]+1
cells

In [ ]:
np.arange(1, 1+len(cells)).reshape((-1,1))

In [ ]:
np.hstack((np.arange(1, 1+len(cells)).reshape((-1,1)), cells)).shape